In [3]:
import os
from langchain_community.document_loaders import JSONLoader

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import json
from pathlib import Path
from pprint import pprint
from srn_data_collector.parse_esrs_requirements.parse_esrs_for_rag import parse_data
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp


In [4]:
file_pth = open('/root/.config/openai.txt')
ls_file = open('/root/.config/langsmith.txt')
os.environ["OPENAI_API_KEY"] = file_pth.read().strip()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ls_file.read().strip()

#cuBLAS settings for llama inference
os.environ["CMAKE_ARGS"] = "-DLLAMA_BLAS=on -DLLAMA_BLAS_VENDOR=OpenBLAS"
os.environ["FORCE_CMAKE"] = "1"

In [2]:
esrs_parsed_json = '/cluster/home/repo/my_llm_experiments/esrs_data_collection/srn_data_collector/parse_esrs_requirements/esrs_requirement_main.json'
esrs_sample_json = '/cluster/home/repo/my_llm_experiments/esrs_data_collection/srn_data_collector/parse_esrs_requirements/query_req_data.json'
recommendation_response_path = '/cluster/home/srn_storage/sample_retriever_recommendations'
sample_pdf_raw_parsed_json_path = '/cluster/home/srn_storage/7aa3d012-661d-4953-8516-30c3cf87fb33/2022/AR/fff9295a-2d30-468c-873c-aa4e6c5d7f14.json'
sample_pdf_formatted_path = '/cluster/home/repo/my_llm_experiments/esrs_data_collection/srn_data_collector/parse_esrs_requirements/docu_data.json'

### Loading requirements into vectorDB

In [7]:
# esrs_parsed = parse_data(esrs_parsed_json)
# with open(esrs_sample_json, 'w') as f:
#     json.dump(esrs_parsed,f)

# esrs_parsed

# esrs_parsed.keys()

# main_categories_map = {"environmental": ['climate change', 'pollution', 'water and marine resources', 'biodiversity and ecosystems'],
#                       "social": ['circular economy', 'own workforce', 'workers in the value chain', 'affected communities', 'consumers and end-users'],
#                       "global" : ["business conduct"]}

# def q_builder_metadata_func(record: dict, metadata: dict) -> dict:

#     metadata["label"] = record.get("label")

#     return metadata

# loader = JSONLoader(
#     file_path=esrs_sample_json,
#     jq_schema=' ."climate change"[], ."own workforce"[], ."business conduct"[]',
#     text_content=False,
#     content_key='text',
#     metadata_func=q_builder_metadata_func)

# query_data = loader.load()

# len(query_data)

# query_data[0].page_content

# query_data[0].metadata

# # text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# # splits_query = text_splitter.split_documents(query_data)
# # vectorstore_query = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
# vectorstore_query = Chroma.from_documents(documents=query_data, embedding=OpenAIEmbeddings())

# # Retrieve and generate using the relevant snippets of the blog.
# query_retriever = vectorstore_query.as_retriever(search_type="similarity", search_kwargs={"k": 6})

# query_retriever.invoke("Query about Climate change, GHG emissions")[:5]

### Loading entire document in vectorDB

In [20]:
with open(sample_pdf_raw_parsed_json_path, 'r') as f:
    sample_doc_dict = json.load(f)

In [21]:
doc_formatted_data = {"doc_data" : []}
for doc_ref, blob_list in sample_doc_dict.items():
    for blob_id, blob_data in enumerate(blob_list):
        doc_formatted_data['doc_data'].append({"text": blob_data['text'], "blob_id": blob_id, "doc_ref": doc_ref})

In [22]:
with open(sample_pdf_formatted_path, 'w') as f:
    json.dump(doc_formatted_data, f)

In [23]:
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["blob_id"] = record.get("blob_id")
    metadata["doc_ref"] = record.get("doc_ref")

    # if "source" in metadata:
    #     source = metadata["source"].split("/")
    #     source = source[source.index("langchain"):]
    #     metadata["source"] = "/".join(source)

    return metadata

In [1]:
doc_loader = JSONLoader(
    file_path=sample_pdf_formatted_path,
    jq_schema='.doc_data[]',
    text_content=False,
    content_key='text',
    metadata_func=metadata_func)

doc_data = doc_loader.load()

NameError: name 'JSONLoader' is not defined

In [25]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits_doc = text_splitter.split_documents(doc_data)
# vectorstore_doc = Chroma.from_documents(documents=splits_doc, embedding=OpenAIEmbeddings())
vectorstore_doc = Chroma.from_documents(documents=doc_data, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever_doc = vectorstore_doc.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [68]:
doc_data[0]

Document(page_content='Connect. \nGrow.\nThrive.\n2022 Annual Report', metadata={'source': '/cluster/home/repo/my_llm_experiments/esrs_data_collection/srn_data_collector/parse_esrs_requirements/docu_data.json', 'seq_num': 1, 'blob_id': 0, 'doc_ref': '1'})

In [7]:
# Creating a prompt template 
sample_requirement = """
AR 43. When preparing the information on gross Scope 2 GHG emissions required under 
paragraph 46, the undertaking shall:
(a) consider the principles and requirements of the GHG Protocol Scope 2 Guidance 
(version 2015 or the latest one); it may also consider corresponding requirements 
for the quantification of indirect GHG emissions from imported energy in ISO 
14064-1:2018;
(b) include purchased or acquired electricity, steam, heat, and cooling consumed by 
the undertaking; 
(c) avoid double counting of GHG emissions reported under Scope 1 or 3;
(d) apply the location-based and market-based methods to calculate Scope 2 GHG 
emissions;
Note: Location-based method quantifies Scope 2 GHG emissions based on 
average energy generation emission factors for defined locations, including local, 
subnational, or national boundaries (GHG Protocol, �Scope 2 Guidance�, 
Glossary, 2015);
Note: Market-based method quantifies Scope 2 GHG emissions based on GHG 
emissions emitted by the generators from which the reporting entity contractually 
purchases electricity bundled with instruments, or unbundled instruments on their 
own (GHG Protocol, �Scope 2 Guidance�, Glossary, 2015); in this case, the 
undertaking may disclose the share of market-based scope 2 GHG emissions 
linked to purchased electricity bundled with instruments such as Guarantee of 
Origins or Renewable Energy Certificates.
(e) disclose biogenic emissions of carbon from the combustion or biodegradation of 
biomass separately from the Scope 2 GHG emissions but include emissions of 
other types of GHG (in particular N2O). In case the emission factors applied do 
not separate the percentage of biomass or biogenic CO2, the undertaking shall 
disclose this. In case GHG emissions other than CO2 (particularly N2O) are not 
available for, or excluded from, location-based grid average emissions factors or 
with the market-based method information, the undertaking shall disclose this;
(f) exclude any purchased, sold or transferred carbon credits or GHG allowances 
from the calculation of Scope 2 GHG emissions; 
(g) adhere to the rules as set out in chapter 7.1 of the GHG Protocol Scope 2 
Guidance (version 2015 or the latest one) and disclose the required information 
accordingly; 
(h) disclose carbon uptakes and emissions (CO2, CO, CH4) from indirect land use 
and land use change separately from the Scope 2 GHG emissions, but include 
emissions of other types of GHG when applicable.
"""

# template = '''
# As an expert in sustainability reports and the new CSRD directive,
# your task is to identify relevant paragraph ids from a list of given
# paragraphs based on a given regulatory Requirement.
# Rank the passages below based on their relevance to the Requirement
# and Sub-Requirement in decreasing order. All the passages
# should be included and listed using identifiers, in
# descending order of relevance. The output format
# should be [] > [], e.g., [1] > [2] > [0]. Only respond
# with the ranking results, do not say any word or explain.
# Requirement: {sample_requirement}
# Passages (with their respective passage ID): 
# {passage_data}
# Response:
# '''

template = '''
[INST] <<SYS>>
As an expert in sustainability reports and the new CSRD directive,
your task is to identify relevant paragraph ids from a list of given
paragraphs based on a given regulatory Requirement.
Rank the passages below based on their relevance to the Requirement
and in decreasing order. All the passages
should be included and listed using identifiers, in
descending order of relevance. The output format
should be [] > [], e.g., [1] > [2] > [0]. Only respond
with the ranking results, do not say any word or explain.
<</SYS>>
Requirement: {sample_requirement}
Passages: {passage_data}
Response:
[/INST]
'''

In [8]:
custom_rag_prompt = PromptTemplate.from_template(template)

In [11]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(f"[{doc_idx}]: {doc.page_content}" for doc_idx, doc in enumerate(docs))


rag_chain = (
    {"passage_data": waliml_retriever | format_docs, "sample_requirement": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [23]:
type(rag_chain)

langchain_core.runnables.base.RunnableSequence

In [12]:
# for chunk in rag_chain.invoke(sample_requirement):
#     print(chunk, end="", flush=True)

rag_chain.invoke("E1.AR43")

processing recommendations for gross scope 2 GHG emissions (market-based), change to prior fy in %: 100%|█| 5/5 [00:00<00:00, 9822.
processing recommendations for total gross GHG emissions (market-based), change to prior fy in %: 100%|█| 5/5 [00:00<00:00, 10613.1
processing recommendations for total gross scope 3 GHG emissions, fy/fy-1 in %: 100%|██████████████| 5/5 [00:00<00:00, 9901.57it/s]
processing recommendations for gross scope 1 GHG emissions, change to prior fy in %: 100%|████████| 5/5 [00:00<00:00, 17697.49it/s]


'[3] > [9] > [4] > [11] > [1] > [6] > [13] > [14] > [16] > [17] > [18] > [19] > [0] > [2] > [5] > [7] > [8] > [10] > [12] > [15]'

In [14]:
retriever_doc.invoke("GHG emissions")

[Document(page_content='GHG Emissions:', metadata={'blob_id': 4, 'doc_ref': '36', 'seq_num': 270, 'source': '/cluster/home/repo/my_llm_experiments/esrs_data_collection/srn_data_collector/parse_esrs_requirements/docu_data.json'}),
 Document(page_content='Science-Based GHG Target', metadata={'blob_id': 5, 'doc_ref': '35', 'seq_num': 259, 'source': '/cluster/home/repo/my_llm_experiments/esrs_data_collection/srn_data_collector/parse_esrs_requirements/docu_data.json'}),
 Document(page_content='Scope 1 emissions include fugitive emissions using emission factors from the “Refrigerant & other” worksheet in the condensed set of the 2021 UK GHG Conversion factors set. \nEmissions from fuel (motor diesel and gasoline) are calculated using the World Resources Institute (2015) GHG Protocol Tool for Stationary Combustion (version 4.1) and the \nMobile Combustion GHG Emissions Calculation Tool (version 2.6).\nScope 2 emissions (market-based method) were calculated using Electricity Emission Factors f

In [4]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from typing import List


class CustomRetriever(BaseRetriever):
    
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        return [Document(page_content=query)]

retriever = CustomRetriever()

retriever.get_relevant_documents("bar")

[Document(page_content='bar')]

In [ ]:
retriever.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [5]:
from typing import Dict
import json
from tqdm import tqdm

class WaliMLRetriever(BaseRetriever):
    retriever_config: Dict

    def _get_relevant_documents(
        self, query: str,* , run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        result = []
        recommended_doc_dict = self.custom_retriever()
        for compliance_item in recommended_doc_dict:
            if recommended_doc_dict[compliance_item]["section_name"] == query:
                recommendations = recommended_doc_dict[compliance_item].get('recommendations', [])
                for recommendation in tqdm(recommendations, desc=f"processing recommendations for {compliance_item}"):
                    blob_data, _ = recommendation
                    result.append(Document(page_content=blob_data.pop("text"),
                                            metadata=blob_data))

        return result
    def custom_retriever(self):
        if "debug_doc" in self.retriever_config:
            with open(self.retriever_config['debug_doc']['path'], 'r') as f:
                recommended_doc_dict = json.load(f)
        else:
            pass
            # with open(self.retriever_config["wali_ml_training_pkldocs"], 'rb') as f:
            #     training_docs = pkl.load(f)
            # validation_doc_list = [doc["id_"] for doc in training_docs 
            #                     if doc['split'].name == "VALIDATION"]
        return recommended_doc_dict

In [6]:
retriever_config = {"debug_doc": {"path": "/cluster/home/srn_recommendation_results/fff9295a-2d30-468c-873c-aa4e6c5d7f14.json"}}
waliml_retriever = WaliMLRetriever(retriever_config=retriever_config)
waliml_retriever._get_relevant_documents("E1.AR43",run_manager=None)

processing recommendations for gross scope 2 GHG emissions (market-based), change to prior fy in %: 100%|█| 5/5 [00:00<00:00, 8694.
processing recommendations for total gross GHG emissions (market-based), change to prior fy in %: 100%|█| 5/5 [00:00<00:00, 17985.8
processing recommendations for total gross scope 3 GHG emissions, fy/fy-1 in %: 100%|█████████████| 5/5 [00:00<00:00, 12858.07it/s]
processing recommendations for gross scope 1 GHG emissions, change to prior fy in %: 100%|████████| 5/5 [00:00<00:00, 19544.75it/s]


[Document(page_content='Energy use\nKPI\n2020\n2021\n2022\nBase station and fixed network sites\nEnergy from fuel (MWh)\n \n46,721  \n46,590  \n50,046 \nGrid electricity (MWh)\n \n459,496  \n514,684  \n532,301 \nOur fleet\nEnergy from fuel (MWh)\n \n53,630  \n52,017  \n45,803 \nData centers and offices\nEnergy from fuel (MWh)\n \n3,220  \n2,281  \n2,350 \nGrid electricity (MWh)\n \n124,808  \n118,679  \n131,975 \nShops\nEnergy from fuel (MWh)\n \n626  \n105  \n109 \nGrid electricity (MWh)\n \n16,538  \n13,304  \n14,401 \nTotal energy consumption (MWh)\nGrid electricity (MWh)\n \n600,842  \n646,667  \n678,677 \nEnergy from fuel (MWh)\n \n104,197  \n100,993  \n98,308 \nTotal energy consumption (MWh)\n \n705,039  \n747,660  \n776,985 \nOut of which energy from renewable sources (MWh)\nNew KPI for 2021  \n18,772  \n28,208 \nScope 1 emissions (Tonnes of CO2e)\n \n33,629  \n33,161  \n31,942 \nScope 2 emissions (Tonnes of CO2e)\n \n152,060  \n146,525  \n139,242 \nScope 3 emissions (Tonnes of 

In [9]:
def format_docs(docs):
    return "\n\n".join(f"[{doc_idx}]: {doc.page_content}" for doc_idx, doc in enumerate(docs))


llama_llm = LlamaCpp(
    model_path="/cluster/home/repo/my_llm_experiments/llama-weights/13B/llama-2-13b-chat.ggufv3.q8_0.bin",
    n_gpu_layers=-1,
    n_batch=512,
    n_ctx=5000,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

# llm("The first man on the moon was ... Let's think step by step")


local_rag_chain = (
    {"passage_data": waliml_retriever | format_docs, "sample_requirement": RunnablePassthrough()}
    | custom_rag_prompt
    | llama_llm
    | StrOutputParser()
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 8 CUDA devices:
  Device 0: Tesla V100-SXM2-32GB, compute capability 7.0, VMM: yes
  Device 1: Tesla V100-SXM2-32GB, compute capability 7.0, VMM: yes
  Device 2: Tesla V100-SXM2-32GB, compute capability 7.0, VMM: yes
  Device 3: Tesla V100-SXM2-32GB, compute capability 7.0, VMM: yes
  Device 4: Tesla V100-SXM2-32GB, compute capability 7.0, VMM: yes
  Device 5: Tesla V100-SXM2-32GB, compute capability 7.0, VMM: yes
  Device 6: Tesla V100-SXM2-32GB, compute capability 7.0, VMM: yes
  Device 7: Tesla V100-SXM2-32GB, compute capability 7.0, VMM: yes
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /cluster/home/repo/my_llm_experiments/llama-weights/13B/llama-2-13b-chat.ggufv3.q8_0.bin (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:          

In [13]:
x = local_rag_chain.invoke("E1.AR43")

processing recommendations for gross scope 2 GHG emissions (market-based), change to prior fy in %: 100%|█| 5/5 [00:00<00:00, 14810
processing recommendations for total gross GHG emissions (market-based), change to prior fy in %: 100%|█| 5/5 [00:00<00:00, 19099.7
processing recommendations for total gross scope 3 GHG emissions, fy/fy-1 in %: 100%|█████████████| 5/5 [00:00<00:00, 11143.21it/s]
processing recommendations for gross scope 1 GHG emissions, change to prior fy in %: 100%|████████| 5/5 [00:00<00:00, 20500.02it/s]

Based


Llama.generate: prefix-match hit


 on the provided passages, I have ranked them based on their relevance to E1.AR43 as follows:

[1] > [2] > [3] > [4] > [5] > [6] > [7] > [8] > [9] > [10] > [11] > [12] > [13] > [14] > [15] > [16] > [17] > [18] > [19]

The most relevant passages are:

[1] > [2] > [3] > [4] > [5] > [6] > [7] > [8] > [9] > [10] > [11] > [12] > [13] > [14] > [15] > [16] > [17] > [18] > [19]

Note: The ranking is based on the relevance to E1.AR43, not necessarily on the importance or popularity of the passage.


llama_print_timings:        load time =     334.26 ms
llama_print_timings:      sample time =     119.95 ms /   243 runs   (    0.49 ms per token,  2025.93 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    8519.28 ms /   243 runs   (   35.06 ms per token,    28.52 tokens per second)
llama_print_timings:       total time =    9595.29 ms /   244 tokens


In [17]:
x

'Based on the provided passages, I have ranked them based on their relevance to E1.AR43 as follows:\n\n[1] > [2] > [3] > [4] > [5] > [6] > [7] > [8] > [9] > [10] > [11] > [12] > [13] > [14] > [15] > [16] > [17] > [18] > [19]\n\nThe most relevant passages are:\n\n[1] > [2] > [3] > [4] > [5] > [6] > [7] > [8] > [9] > [10] > [11] > [12] > [13] > [14] > [15] > [16] > [17] > [18] > [19]\n\nNote: The ranking is based on the relevance to E1.AR43, not necessarily on the importance or popularity of the passage.'

In [32]:
wali_ml_training_pkldocs = "/cluster/home/training_runs/esrs_classifier_final/Parsing/ordinary-priority-000/documents.p"

In [37]:
import pickle as pkl

with open(wali_ml_training_pkldocs, 'rb') as f:
    training_docs = pkl.load(f)

/root/miniconda3/envs/lit-llama/lib/python3.11/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_inputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [50]:
training_docs[0]['id_']

'95df1d10-26ac-4bc2-a4b5-41f1698cac24'

In [51]:
sample_rec_doc = "/cluster/home/srn_recommendation_results/fff9295a-2d30-468c-873c-aa4e6c5d7f14.json"

In [52]:
with open(sample_rec_doc, 'r') as f:
    rec_doc_dict = json.load(f)

In [56]:
rec_doc_dict['gross scope 2 GHG emissions (market-based), change to prior fy in %'].keys()

dict_keys(['recommendations', 'ground_truth', 'section_name', 'section_name_text'])

In [64]:
rec_doc_dict['gross scope 2 GHG emissions (market-based), change to prior fy in %']['section_name']

'E1.AR43'